In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.datasets import make_classification

# 1. Memuat Data
try:
    df = pd.read_csv('dataset_flora_fauna.csv')
    print("Dataset berhasil dimuat.")
    print("Info dataset:")
    df.info()
    print("\nJumlah sampel per kelas sebelum SMOTE:")
    print(df['Tipe Spesies'].value_counts())
except FileNotFoundError:
    print("Error: Pastikan file 'dataset_flora_fauna.csv' berada di direktori yang sama.")
    # Membuat dataframe dummy jika file tidak ditemukan agar sisa kode bisa berjalan
    X, y_dummy = make_classification(n_samples=50000, n_features=10, n_informative=5, n_redundant=0, n_classes=2, random_state=42)
    df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(10)])
    df['Tipe Spesies'] = y_dummy
    print("\nMembuat dataset dummy karena file asli tidak ditemukan.")


# 2. Preprocessing
# Memisahkan fitur (X) dan target (y)
# Menghapus kolom 'Spesies' karena memiliki terlalu banyak nilai unik (high cardinality)
# yang tidak cocok untuk one-hot encoding dan dapat menyebabkan MemoryError.
X = df.drop(['Tipe Spesies', 'Spesies'], axis=1)
y = df['Tipe Spesies']

# Mengatasi fitur kategorikal jika ada (contoh: menggunakan one-hot encoding)
# Diasumsikan semua fitur selain target adalah numerik. Jika tidak, lakukan encoding.
X = pd.get_dummies(X, drop_first=True).astype(int)

# Mengubah label target menjadi numerik
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 3. Split Data
# Memastikan jumlah data cukup untuk split 40000/10000
if len(df) < 50000:
    print(f"\nPeringatan: Jumlah data hanya {len(df)}, tidak cukup untuk split 40000/10000.")
    print("Split akan disesuaikan dengan rasio 80/20.")
    test_size = 0.2
    train_size = None
else:
    train_size = 40000
    test_size = 10000

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded,
    train_size=train_size,
    test_size=test_size,
    random_state=42,
    stratify=y_encoded # Menjaga proporsi kelas pada data train dan test
)

print(f"\nUkuran data training: {X_train.shape[0]}")
print(f"Ukuran data testing: {X_test.shape[0]}")


# 4. Pemodelan dengan SMOTE dan 10-Fold Cross-Validation
# Mendefinisikan model-model yang akan digunakan
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'KNN': KNeighborsClassifier()
}

# Cross-validation setup
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results = {}

print("\nMemulai evaluasi model dengan 10-Fold Cross-Validation dan SMOTE...")

for name, model in models.items():
    # Membuat pipeline yang menggabungkan SMOTE, Scaling, dan Model
    # Scaling penting untuk Logistic Regression dan KNN
    pipeline = ImbPipeline(steps=[
        ('smote', SMOTE(random_state=42)),
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])
    
    # Melakukan cross-validation pada data training
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
    results[name] = scores
    print(f"{name}: Akurasi CV = {scores.mean():.4f} (Std: {scores.std():.4f})")

# 5. Melatih model terbaik pada seluruh data training dan evaluasi pada data testing
best_model_name = max(results, key=lambda name: results[name].mean())
best_model = models[best_model_name]

print(f"\nModel terbaik berdasarkan Cross-Validation adalah: {best_model_name}")
print("Melatih model terbaik pada seluruh data training (dengan SMOTE)...")

# Membuat pipeline final untuk model terbaik
final_pipeline = ImbPipeline(steps=[
    ('smote', SMOTE(random_state=42)),
    ('scaler', StandardScaler()),
    ('classifier', best_model)
])

# Melatih pipeline pada data training
final_pipeline.fit(X_train, y_train)

# Evaluasi pada data testing
y_pred = final_pipeline.predict(X_test)

print("\nLaporan Klasifikasi pada Data Testing:")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print(f"Akurasi pada Data Testing: {accuracy_score(y_test, y_pred):.4f}")

Dataset berhasil dimuat.
Info dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Spesies                       50000 non-null  object
 1   Tipe Spesies                  50000 non-null  object
 2   Ukuran                        50000 non-null  object
 3   Habitat                       50000 non-null  object
 4   Iklim Habitat                 50000 non-null  object
 5   Jenis Makanan                 50000 non-null  object
 6   Interaksi Manusia             50000 non-null  object
 7   Status Konservasi             50000 non-null  object
 8   Adaptasi terhadap Lingkungan  50000 non-null  object
 9   Penyebab Ancaman              50000 non-null  object
 10  Lama Bertahan Hidup           50000 non-null  object
dtypes: object(11)
memory usage: 4.2+ MB

Jumlah sampel per kelas sebelum SMOTE:
Tipe Spesies
faun